# Sales Data Automation with Google Sheets

## Scenario
You're a sales operations analyst at a growing company. Every week, you receive a sales report with 50+ transactions. Your job is to:
- Calculate commissions for each sale
- Flag data quality issues (missing information, negative quantities)
- Clean and prepare the data for management review

**Currently:** This takes 30-60 minutes of manual work in Excel.

**After this tutorial:** You'll automate the entire process to run in under 2 minutes.

## What You'll Build
By the end of this notebook, you'll have a Python script that:
1. Connects to a Google Sheet containing raw sales data
2. Calculates commissions automatically (Subtotal × 8% commission rate)
3. Applies business logic to flag data quality issues
4. Writes the processed data back to Google Sheets

**No coding experience required** - just follow along and run each cell in order!

In [ ]:
# SECTION 1: Setup and Authentication
# This cell authenticates your Google Colab session to access Google Sheets

# Import required libraries
from google.colab import auth  # Handles Google authentication
import gspread  # Library for working with Google Sheets
from google.auth import default  # Gets your credentials

# Authenticate with Google
# This will open a popup asking you to sign in and grant permissions
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("✓ Successfully authenticated!")
print("You can now access Google Sheets from this notebook.")

## Instructions: Prepare Your Data

Before continuing, you need to upload the sales data to Google Sheets:

### Step-by-step:
1. **Download** the `sales_data_raw.csv` file from this repository
2. **Go to** [Google Sheets](https://sheets.google.com)
3. **Create** a new blank spreadsheet
4. **Import the CSV:**
   - Click `File` → `Import` → `Upload`
   - Select `sales_data_raw.csv`
   - Choose "Replace spreadsheet" and click "Import data"
5. **Share the sheet:**
   - Click the green `Share` button (top right)
   - Click "Change to anyone with the link"
   - Set permissions to **Editor**
   - Click `Copy link`
6. **Paste the URL** into the next code cell (where it says `PASTE_YOUR_URL_HERE`)

**Why does this matter?** Google Colab needs permission to read and write to your spreadsheet. By sharing with "anyone with the link," your authenticated session can access it.

In [ ]:
# SECTION 2: Connect to Your Google Sheet
# TODO: Paste your Google Sheet URL between the quotes below
SHEET_URL = "PASTE_YOUR_URL_HERE"

# Open the spreadsheet using the URL
spreadsheet = gc.open_by_url(SHEET_URL)

# Get the first worksheet (Sheet1)
worksheet = spreadsheet.sheet1

# Read all data into a list of dictionaries
# Each row becomes a dictionary where column headers are keys
data = worksheet.get_all_records()

print(f"✓ Successfully loaded {len(data)} transactions")
print(f"\nSpreadsheet name: {spreadsheet.title}")

In [ ]:
# Let's preview the data to understand what we're working with
import pandas as pd

# Convert to pandas DataFrame for easier viewing
df = pd.DataFrame(data)

print("First 3 transactions:")
print(df.head(3))

print(f"\nColumns in the dataset:")
print(list(df.columns))

## Understanding the Data

Let's break down what each column means:

- **Transaction_ID**: Unique identifier for each sale (e.g., T0001, T0002)
- **Date**: When the sale occurred
- **Sales_Rep**: Name of the salesperson who made the sale
- **Region**: Geographic region (West, East, Midwest, etc.)
- **Product**: What was sold
- **Unit_Price**: Price per item
- **Quantity**: Number of items sold
- **Subtotal**: Total sale amount (Unit_Price × Quantity)
- **Commission_Rate**: Percentage commission (0.08 = 8%)
- **Commission**: Commission earned - **EMPTY (we'll calculate this!)**
- **Status**: Data quality status - **EMPTY (we'll assign this!)**

### What You'll Notice
This is **realistic messy data**:
- Commission column is blank (needs calculation)
- Status column is blank (needs business logic)
- Some Region values are missing
- Some Quantity values are negative (data entry errors!)
- Some sales rep names are lowercase (formatting issues)
- Dates are in mixed formats

**This is normal!** Real-world data is never perfect. Let's automate fixing it.

---
# SECTION 3: Calculate Commissions

## Task 1: Calculate Commissions

Every sales rep earns an 8% commission on their sales.

**Formula:** `Commission = Subtotal × Commission_Rate`

**Example:**
- Subtotal: $209.93
- Commission Rate: 0.08 (which is 8%)
- Commission: $209.93 × 0.08 = **$16.79**

We'll loop through each transaction and calculate this automatically.

In [ ]:
# Calculate commission for each transaction
for row in data:
    # Get the subtotal and commission rate for this transaction
    subtotal = row['Subtotal']
    commission_rate = float(row['Commission_Rate'])
    
    # TODO: Calculate commission (subtotal × commission_rate)
    commission = subtotal * commission_rate
    
    # Store the result back in the row (rounded to 2 decimal places)
    row['Commission'] = round(commission, 2)

print("✓ Commissions calculated for all transactions")
print(f"\nExample: Transaction {data[0]['Transaction_ID']}")
print(f"  Subtotal: ${data[0]['Subtotal']}")
print(f"  Commission: ${data[0]['Commission']}")

In [ ]:
# Verify: Show first 5 transactions with their calculated commissions
df = pd.DataFrame(data)
print("Commissions calculated:")
print(df[['Transaction_ID', 'Sales_Rep', 'Subtotal', 'Commission']].head())

---
# SECTION 4: Flag Data Quality Issues

## Task 2: Assign Status Codes

Not all data is perfect. We need to flag transactions that have problems so the sales team can fix them.

### Business Rules:
1. **If Quantity is negative** → `ERROR - Negative Quantity`
   - Why? You can't sell -5 items. This is a data entry mistake.
2. **If Region is blank** → `ERROR - Missing Region`
   - Why? We need region data for territory analysis.
3. **Otherwise** → `Complete`
   - The transaction is good to go!

### Why This Matters:
- **Manual approach:** You'd have to look at all 50 rows by eye
- **Automated approach:** The script catches errors instantly
- **At scale:** Works the same for 50 or 5,000 rows

In [ ]:
# Assign status codes based on data quality
error_count = 0

for row in data:
    # Get the values we need to check
    quantity = row['Quantity']
    region = row['Region']
    
    # TODO: Write if/elif/else logic to assign status
    # Check for negative quantities first (highest priority error)
    if quantity < 0:
        row['Status'] = "ERROR - Negative Quantity"
        error_count += 1
    # Check for missing region
    elif region == "" or region is None:
        row['Status'] = "ERROR - Missing Region"
        error_count += 1
    # If no problems, mark as complete
    else:
        row['Status'] = "Complete"

print("✓ Status codes assigned to all transactions")
print(f"\nFound {error_count} errors that need attention")

In [ ]:
# Show all transactions with errors
df = pd.DataFrame(data)
errors = df[df['Status'].str.contains('ERROR')]

print(f"Transactions needing attention ({len(errors)} total):")
print(errors[['Transaction_ID', 'Sales_Rep', 'Region', 'Quantity', 'Status']])

## Why This Matters: Manual vs. Automated

### Manual Process:
1. Open Excel
2. Scroll through all 50 rows
3. Look for blank Region cells
4. Look for negative Quantity values
5. Manually type error messages
6. **Time: 15-20 minutes** (and you might miss some!)

### Automated Process:
1. Run the code cell above
2. **Time: 0.5 seconds**
3. **Accuracy: 100%** (never misses an error)

### At Scale:
- 50 rows → saved 20 minutes
- 500 rows → saved 3+ hours
- 5,000 rows → **saved multiple days of work**

This is the power of automation!

---
# SECTION 5: Write Results Back to Google Sheets

## Final Step: Update the Spreadsheet

We've calculated commissions and flagged errors in Python. Now let's write everything back to Google Sheets so the sales team can see the results.

**What happens:**
1. We convert our data to the format Google Sheets expects (list of lists)
2. We clear the old data from the sheet
3. We write the headers + all updated rows
4. The Commission and Status columns are now filled in!

In [ ]:
# Write the processed data back to Google Sheets

# Step 1: Get the column headers from the first row
headers = list(data[0].keys())

# Step 2: Convert each row from dictionary to list of values
# Google Sheets needs data as a list of lists, not dictionaries
rows = []
for row in data:
    # Extract values in the same order as headers
    row_values = [row[col] for col in headers]
    rows.append(row_values)

# Step 3: Clear the existing data in the worksheet
worksheet.clear()

# Step 4: Write headers and all data rows
# [headers] creates a list containing the headers as the first row
# + rows adds all the data rows after it
worksheet.update('A1', [headers] + rows)

print("✓ Data successfully written back to Google Sheet!")
print(f"\nUpdated {len(data)} transactions")
print("\nGo check your spreadsheet:")
print("- Commission column should now be filled")
print("- Status column should show 'Complete' or 'ERROR - ...'")

## What You Just Built

### Summary
You've created an automated sales data processing pipeline that:
1. ✓ Connects to Google Sheets using authentication
2. ✓ Reads 50 sales transactions into Python
3. ✓ Calculates commissions (Subtotal × 8%)
4. ✓ Applies business logic to flag data quality issues
5. ✓ Writes cleaned data back to Google Sheets

### Time Savings
- **Manual process:** 30-60 minutes per week
- **Automated process:** 2 minutes (just run this notebook)
- **Annual savings:** ~40 hours per year

### Reusability
Next week when you get new sales data:
1. Upload the new CSV to Google Sheets
2. Update the `SHEET_URL` variable
3. Run all cells
4. Done!

**No code changes needed** - the same script works on any sales data with the same column structure.

### Business Value
- **Faster:** Process data in minutes instead of hours
- **More accurate:** No human errors in calculations
- **Scalable:** Works for 50 or 5,000 rows
- **Auditable:** Everyone can see exactly what logic was applied

This is a real example of how business analysts use Python to add value!

---
# SECTION 6: Extension Challenges (Optional)

## Want to Go Further?

If you finish early or want to practice more, try these challenges:

### Challenge 1: Sales by Region (Easiest)
Calculate the total sales for each region. Which region has the highest sales?

**Hint:** Use a dictionary to track totals for each region. Skip rows where region is blank.

### Challenge 2: Top Performer (Medium)
Which sales rep earned the most in total commissions?

**Hint:** Similar to Challenge 1, but group by Sales_Rep instead of Region.

### Challenge 3: Error Report Sheet (Advanced)
Create a second worksheet called "Errors" that contains only the transactions with status errors.

**Hint:** You'll need to:
1. Filter `data` to only include rows where Status contains 'ERROR'
2. Create a new worksheet: `error_sheet = spreadsheet.add_worksheet(title="Errors", rows=100, cols=11)`
3. Write the filtered data to that sheet

---

### Starter Code for Challenge 1:

In [ ]:
# CHALLENGE 1: Calculate total sales by region

region_totals = {}

for row in data:
    region = row['Region']
    subtotal = row['Subtotal']
    
    # TODO: Add logic to sum subtotals by region
    # Skip rows where region is blank
    
    if region and region != "":
        if region not in region_totals:
            region_totals[region] = 0
        region_totals[region] += subtotal

# Display results sorted by region name
print("Total Sales by Region:")
print("-" * 40)
for region, total in sorted(region_totals.items()):
    print(f"{region:15} ${total:>12,.2f}")

# Find the top region
top_region = max(region_totals, key=region_totals.get)
print("-" * 40)
print(f"Top Region: {top_region} with ${region_totals[top_region]:,.2f}")

---
# Instructor Notes

## How to Reuse This Notebook Next Week

When you receive new sales data:
1. Upload the new CSV to a new Google Sheet
2. Share the sheet ("Anyone with the link can edit")
3. Update the `SHEET_URL` variable in the code
4. Run all cells (`Runtime` → `Run all`)

**That's it!** The same code will process any sales data with matching column names.

## To Process Different Data

This script will work with any dataset **as long as the column names match**:
- Transaction_ID
- Date
- Sales_Rep
- Region
- Product
- Unit_Price
- Quantity
- Subtotal
- Commission_Rate
- Commission
- Status

If your data has different columns, you'll need to update the field names in the code.

## Common Issues & Solutions

**"Authentication failed"**
- Make sure you're signed into your Google account
- Run the authentication cell again

**"Cannot find spreadsheet"**
- Check that the URL is correct
- Make sure the sheet is shared with "Anyone with the link can edit"

**"KeyError: 'Region'" or similar**
- Column names in your sheet don't match the code
- Check for extra spaces in column headers
- Make sure you imported the CSV correctly (headers in row 1)

---

## Questions?

Feel free to experiment! You can always re-run cells to try different approaches. The worst that can happen is you get an error message - and errors are how we learn.

Happy automating! 🚀